In [88]:
import requests
import pandas as pd
import pprint
import ssl
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import os
import time
from requests.exceptions import ReadTimeout

In [11]:
# importando bibliotecas conexão PDGT
import json
import argparse
import subprocess
import boto3
import time
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions
from scipy.stats import chi2_contingency

In [12]:
class CustomException(Exception):
    pass

In [13]:
# athena info

In [14]:
# athena connection

In [15]:
# athena connection function

In [16]:
url = "https://servicodados.ibge.gov.br/api/v1/pesquisas/23" 

#teste 1
#https://servicodados.ibge.gov.br/api/v3/agregados/5436/periodos/202302/variaveis/5932?localidades=N6[all]&classificacao=2[all]

#teste 2
#https://servicodados.ibge.gov.br/api/v3/agregados/1428/periodos/2010/variaveis/1188|1189?localidades=N6[all]&classificacao=2[all]|1[6795]

#pesquisas
#https://servicodados.ibge.gov.br/api/v1/pesquisas

In [17]:
class TLSAdapter(requests.adapters.HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context()
        ctx.set_ciphers("DEFAULT@SECLEVEL=1")
        ctx.options |= 0x4   # <-- the key part here, OP_LEGACY_SERVER_CONNECT
        kwargs["ssl_context"] = ctx
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)

In [18]:
with requests.session() as s:
    s.mount("https://", TLSAdapter())
    response = s.get(url)
    data = response.json()

In [19]:
#df = pd.DataFrame(data)
#df

In [20]:
#df.to_excel('pesquisas_ibge.xlsx', index=False, sheet_name='pesquisas')

In [21]:
pprint.pprint(data)

{'contexto': '101010',
 'descricao': None,
 'id': 23,
 'nome': 'Censo 2010',
 'observacao': None,
 'periodos': [{'fonte': ['IBGE, Censo Demográfico 2010'],
               'nota': [],
               'periodo': '2010',
               'publicacao': '26/09/2016 12:44:19',
               'versao': 23}]}


In [22]:
#for i in range(0, len(informacoes)):
#    currentItem = informacoes[i]
#    df.loc[i] = [data[i]['']]

In [23]:
#teste = pd.DataFrame(informacoes.features.tolist())
#pd.json_normalize(teste).head()

In [71]:
query_endereco = """
sql query
"""

In [72]:
endereco_df = execute_athena_query(query_endereco)

In [73]:
geolocator = Nominatim(user_agent="my_geocoder")

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

minimum_delay = 1

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        start_time = time.time()  # Record the start time
        location1 = geolocator.geocode(row['endereco_paciente'])
        end_time = time.time()
        request_time = end_time - start_time

        time.sleep(max(0, minimum_delay - request_time))

        start_time2 = time.time()
        location2 = geolocator.geocode(row['endereco_unidade'])
        end_time2 = time.time()  # Record the end time
        request_time2 = end_time2 - start_time2

        time.sleep(max(0, minimum_delay - request_time2))

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)

            return geodesic(coords1, coords2).kilometers
        
        else:
            # If either location is None, return None to skip the row
            return None
        
  #  except ReadTimeout:
   #     # Handle ReadTimeoutError by retrying with exponential backoff
   #     print("Timeout error occurred. Retrying...")
   #     time.sleep(2 ** retries)  # Exponential backoff
   #     return calcular_distancia(row)  # Retry the function
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Define a minimum delay between requests (in seconds)
minimum_delay = 1.0

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        start_time = time.time()  # Record the start time
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])
        end_time = time.time()  # Record the end time
        request_time = end_time - start_time

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)
            sleep_time = max(0, minimum_delay - request_time)
            time.sleep(sleep_time)

            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except ReadTimeout:
        # Handle ReadTimeoutError by retrying with exponential backoff
        print("Timeout error occurred. Retrying...")
        time.sleep(2)  # Exponential backoff
        return calcular_distancia(row)  # Retry the function
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

In [ ]:
# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)

            # Sleep for the calculated time
            time.sleep(1)

            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="my_geocoder")

# Function to calculate the distance between two addresses
def calcular_distancia(row):
    try:
        location1 = geolocator.geocode(row['endereco_paciente'])
        location2 = geolocator.geocode(row['endereco_unidade'])

        if location1 and location2:
            coords1 = (location1.latitude, location1.longitude)
            coords2 = (location2.latitude, location2.longitude)
            return geodesic(coords1, coords2).kilometers
        else:
            # If either location is None, return None to skip the row
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Apply the function to calculate the distance and create a 'Distancia' column
endereco_df['distancia'] = endereco_df.apply(calcular_distancia, axis=1)

# Examine the DataFrame with the distance column
endereco_df

In [76]:
endereco_df.isna().mean()*100

id_paciente           0.0
id_unidade            0.0
unidade               0.0
endereco_paciente     0.0
endereco_unidade      1.0
cidade_clinica        0.0
distancia            42.0
dtype: float64

In [77]:
endereco_df.to_excel('distancia_paciente_validacao.xlsx', index=False, sheet_name='base')

In [86]:
teste = geolocator.geocode('RUA DOUTOR JOAQUIM SOARES LELIS, 415, RIBEIRÃO DAS NEVES, SP')
coords1 = (teste.latitude, teste.longitude)
print(teste.latitude)
print(teste.longitude)

AttributeError: 'NoneType' object has no attribute 'latitude'

In [79]:
teste2 = geolocator.geocode('AVENIDA ESPANHA, 563, ARARAQUARA, SP')
coords2 = (teste2.latitude, teste2.longitude)
print(teste2.latitude)
print(teste2.longitude)
#Paulinho Pipas -> R. Dr. José Santos Neto, 54, Jardim Suzanopolis, Suzano, SP
#paciente 56854683 endereço Rua Dr. Aniz Fadul, 1494, Vila Maluf, Suzano, SP, 08685-020
#

-21.792396
-48.17844


In [80]:
distancia = geodesic(coords1, coords2).kilometers
distancia

3.147726721288721